# Import Widget Libraries

In [1]:
%load_ext autoreload   
%autoreload 2
import kyurem
from kyurem.core.idom_loader import reload_bundle
from kyurem.core.widget import WidgetModel

kyurem_client: 0.0.0


In [2]:
from kyurem import Service
service = Service(kh = 'indeed.v0.0.1:612e3403')

## Load data 

In [3]:
import pandas as pd
df1 = pd.read_csv("indeed.v0.0.1_612e3403_sampled.csv")
corpus_data = df1.to_dict('records')

df2 = pd.read_csv("indeed.v0.0.1_612e3403_merged.csv")
merge_data = df2.to_dict('records')

In [4]:
# TODO: pass column na
service.load_corpus(corpus_data, 'parent_title', 'context', 'child_title')

In [5]:
service.load_merge_data(merge_data, 'corpus_entity', 'node_label', 'node_uuid', 'node_title')

In [6]:
decision_list = ["Accept", "Reject", "Defer"]

In [7]:
mergedata = service.get_merge_data()["rows"]

# Create Widget

In [9]:
from kyurem import MergeVerifier


def init(state): 
    # Fetch initial view data
    return {
        "subgraph": None,
        "corpus": None,
        "mergedata":mergedata,
        "decisions":decision_list,
    }

def focus(state, row, panel):
    # node can be in the following format:
    # node : { "node_label": str, "node_property": str, "node_property_value": str }
    node = { "node_label": row["node_label"], "node_property": "title", "node_property_value": row["node"] }
    entity = { "node_label": row["node_label"], "node_property": "title", "node_property_value": row["entity"] }
    # Fetch the neighborhood around the input node 
    result = service.get_node_neighborhood(node)
    datatable = service.get_annotated_corpus(entity)
    
    data = {}
    data["subgraph"] = result["schema"]
    # TODO: Move conversion code into service
    if datatable["rows"]:
        data["corpus"] = datatable["rows"]
        data["highlight"] = datatable["highlight"]
    else:
        data["corpus"] = None
        data["highlight"] = None
    return data
  
# Create widget 
reload_bundle()
merge_verifier = MergeVerifier({
        "init": init,
        "focus": focus
    }, mergedata)

merge_verifier.show()

component(105257dc0, self=<kyurem.widgets.WidgetWithHistory.WidgetWithHistory object at 0x1143d5430>)

LayoutWidget(Layout(component(105257dc0, self=<kyurem.widgets.WidgetWithHistory.WidgetWithHistory object at 0x…

In [ ]:
merge_verifier.state.data.mergedata